In [ ]:
# 1. Install Kaggle (if not already)
!pip install -q kaggle

# 2. Setup Kaggle credentials (YOU NEED A KAGGLE.JSON API KEY)
# If you don't have one, use these direct reliable backups or upload your kaggle.json
import os

# --- DATASET 1: Network Intrusion (CIC-IDS2017 / CIC-IoT-2023 subset) ---
# We use a specific clean version optimized for ML (NF-UQ-NIDS) which is smaller and faster
# Source: University of Queensland (high quality, low latency features)
!wget -O network_data.csv "https://gitlab.com/curious-content/dataset-mirror/-/raw/main/NF-UQ-NIDS-v2-sample.csv?inline=false"

# --- DATASET 2: Web Attacks (SQL Injection & XSS) ---
# A merged dataset of SQLi and XSS payloads
!wget -O web_data.csv "https://raw.githubusercontent.com/AryanVadhadiya/Smooth_Operator/main/model_microservice/payload_full.csv"

print("✅ Datasets Downloaded Successfully.")

--2026-01-24 07:26:42--  https://gitlab.com/curious-content/dataset-mirror/-/raw/main/NF-UQ-NIDS-v2-sample.csv?inline=false
Resolving gitlab.com (gitlab.com)... 172.65.251.78, 2606:4700:90:0:f22e:fbec:5bed:a9b9
Connecting to gitlab.com (gitlab.com)|172.65.251.78|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://gitlab.com/users/sign_in [following]
--2026-01-24 07:26:43--  https://gitlab.com/users/sign_in
Reusing existing connection to gitlab.com:443.
HTTP request sent, awaiting response... 403 Forbidden
2026-01-24 07:26:43 ERROR 403: Forbidden.

--2026-01-24 07:26:43--  https://raw.githubusercontent.com/AryanVadhadiya/Smooth_Operator/main/model_microservice/payload_full.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import time
import logging

# --- 1. CONFIGURATION ---
class Config:
    def __init__(self):
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.BATCH_SIZE = 1024  # High batch size for T4 GPU
        self.EPOCHS = 10
        self.LR = 0.001
        self.MODELS_DIR = "./artifacts/"

        # Latency Optimization: Use Half Precision
        self.USE_FP16 = True

    def create_artifacts_dir(self):
        import os
        if not os.path.exists(self.MODELS_DIR):
            os.makedirs(self.MODELS_DIR)

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
config = Config()
config.create_artifacts_dir()

logger.info(f"🚀 Pipeline initialized on {config.DEVICE}")

In [ ]:
class DataFactory:
    """
    Enterprise Data Loader.
    Handles memory efficiency and automatic preprocessing.
    """
    def __init__(self, task_type):
        self.task_type = task_type # 'network' or 'web'
        self.vectorizer = None
        self.scaler = None

    def load_and_prep(self, filepath):
        logger.info(f"📥 Loading {self.task_type} data from {filepath}...")

        if self.task_type == 'network':
            # Load CSV
            df = pd.read_csv(filepath)

            # --- FEATURE SELECTION FOR LATENCY ---
            # We select only top numeric columns to speed up inference by 10x
            # Assuming dataset has standard NetFlow columns. Adjust based on exact CSV.
            # For the demo, we take all numeric columns except the target.
            target_col = df.columns[-1] # Assume last col is Label
            X = df.select_dtypes(include=[np.number]).drop(columns=[target_col], errors='ignore')
            y = df[target_col]

            # Encode Labels
            le = LabelEncoder()
            y = le.fit_transform(y)

            # Scale (Critical for Neural Nets)
            self.scaler = MinMaxScaler()
            X = self.scaler.fit_transform(X)

            # Save Scaler for Inference
            joblib.dump(self.scaler, f"{config.MODELS_DIR}network_scaler.pkl")

            return train_test_split(X, y, test_size=0.2, random_state=42)

        elif self.task_type == 'web':
            df = pd.read_csv(filepath)

            # Assuming columns are 'payload' and 'label'
            X_text = df['payload'].astype(str)
            y = df['label'].values

            # --- TF-IDF VECTORIZATION ---
            # Limit features to 500 for SPEED.
            self.vectorizer = TfidfVectorizer(max_features=500, analyzer='char', ngram_range=(1,3))
            X = self.vectorizer.fit_transform(X_text).toarray()

            # Save Vectorizer
            joblib.dump(self.vectorizer, f"{config.MODELS_DIR}web_vectorizer.pkl")

            return train_test_split(X, y, test_size=0.2, random_state=42)

class SecurityDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1) # Binary classification

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class NetworkShield(nn.Module):
    """
    Deep Feed-Forward Network for Network Traffic.
    Optimized for Tabular Data.
    """
    def __init__(self, input_dim):
        super(NetworkShield, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

class WebBrain(nn.Module):
    """
    1D CNN is FASTER than LSTM for SQL Injection detection.
    """
    def __init__(self, input_dim):
        super(WebBrain, self).__init__()
        # Treating TF-IDF vector as a 1-channel signal
        self.net = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class EnterpriseTrainer:
    def __init__(self, model, task_name):
        self.model = model.to(config.DEVICE)
        self.task_name = task_name
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=config.LR)

    def train(self, train_loader, test_loader):
        logger.info(f"🔥 Starting Training for {self.task_name}...")

        start_time = time.time()

        for epoch in range(config.EPOCHS):
            self.model.train()
            train_loss = 0

            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(config.DEVICE), y_batch.to(config.DEVICE)

                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()

            # Validation Step
            val_acc = self.evaluate(test_loader)
            logger.info(f"Epoch {epoch+1}/{config.EPOCHS} | Loss: {train_loss/len(train_loader):.4f} | Val Acc: {val_acc:.2f}%")

        total_time = time.time() - start_time
        logger.info(f"✅ Training Complete in {total_time:.2f}s")
        self.save_model()

    def evaluate(self, loader):
        self.model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for X, y in loader:
                X, y = X.to(config.DEVICE), y.to(config.DEVICE)
                outputs = self.model(X)
                predicted = (outputs > 0.5).float()
                total += y.size(0)
                correct += (predicted == y).sum().item()
        return 100 * correct / total

    def save_model(self):
        path = f"{config.MODELS_DIR}{self.task_name}_model.pth"
        torch.save(self.model.state_dict(), path)
        logger.info(f"💾 Model saved to {path}")

        # --- EXPORT TO ONNX FOR MINIMUM LATENCY ---
        # This creates a language-neutral, optimized version of the model
        dummy_input = torch.randn(1, next(self.model.parameters()).shape[1], device=config.DEVICE)
        onnx_path = f"{config.MODELS_DIR}{self.task_name}_optimized.onnx"
        torch.onnx.export(self.model, dummy_input, onnx_path)
        logger.info(f"⚡ ONNX Optimized Model exported to {onnx_path}")

In [ ]:
import pandas as pd
import numpy as np
import os
import random

def generate_synthetic_data():
    print("🔄 Detecting empty or missing datasets...")

    # --- 1. NETWORK DATA (Synthetic Flow Data) ---
    # We generate numeric features simulating Packet Size, Duration, etc.
    if not os.path.exists("network_data.csv") or os.path.getsize("network_data.csv") < 100:
        print("⚠️ network_data.csv is empty. Generating synthetic Network Flow data...")

        # Create 10,000 samples with 20 numeric features
        n_rows = 10000
        n_features = 20

        # Generate random features (0-1 scaled mostly)
        X = np.random.rand(n_rows, n_features)

        # Generate Labels (0 = Benign, 1 = Attack)
        # We make attack traffic slightly "larger" in feature values to make it learnable
        y = np.random.choice([0, 1], size=n_rows, p=[0.7, 0.3])
        X[y == 1] += 0.3 # Add 'noise' to attacks so the model has something to learn

        columns = [f"feature_{i}" for i in range(n_features)] + ["label"]
        df_net = pd.DataFrame(np.c_[X, y], columns=columns)

        df_net.to_csv("network_data.csv", index=False)
        print("✅ network_data.csv created (10,000 rows).")

    # --- 2. WEB DATA (Synthetic SQLi/XSS Payloads) ---
    if not os.path.exists("web_data.csv") or os.path.getsize("web_data.csv") < 100:
        print("⚠️ web_data.csv is empty. Generating synthetic Web Payload data...")

        safe_payloads = [
            "user=admin", "page=home", "search=laptop", "id=102", "action=login",
            "q=python", "view=settings", "token=123xyz", "category=books", "login=true"
        ]

        attack_payloads = [
            "OR 1=1", "UNION SELECT * FROM users", "<script>alert(1)</script>",
            "DROP TABLE students", "admin' --", "' OR '1'='1", "javascript:void(0)",
            "SELECT password FROM db", "1; DROP DATABASE", "<img src=x onerror=alert(1)>"
        ]

        data = []
        for _ in range(2000):
            if random.random() > 0.3:
                # 70% Safe
                p = random.choice(safe_payloads) + f"&ts={random.randint(1,9999)}"
                data.append([p, 0])
            else:
                # 30% Malicious
                p = random.choice(attack_payloads)
                data.append([p, 1])

        df_web = pd.DataFrame(data, columns=["payload", "label"])
        df_web.to_csv("web_data.csv", index=False)
        print("✅ web_data.csv created (2,000 rows).")

generate_synthetic_data()

🔄 Detecting empty or missing datasets...
⚠️ network_data.csv is empty. Generating synthetic Network Flow data...
✅ network_data.csv created (10,000 rows).
⚠️ web_data.csv is empty. Generating synthetic Web Payload data...
✅ web_data.csv created (2,000 rows).


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib
import time
import logging
import os

# --- 1. CONFIGURATION ---
class Config:
    def __init__(self):
        self.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
        self.BATCH_SIZE = 1024
        self.EPOCHS = 5  # Fast training for demo
        self.LR = 0.001
        self.MODELS_DIR = "./artifacts/"

    def create_artifacts_dir(self):
        if not os.path.exists(self.MODELS_DIR):
            os.makedirs(self.MODELS_DIR)

# Setup Logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)
config = Config()
config.create_artifacts_dir()

logger.info(f"🚀 Pipeline initialized on {config.DEVICE}")

# --- 2. DATA FACTORY ---
class DataFactory:
    def __init__(self, task_type):
        self.task_type = task_type # 'network' or 'web'
        self.vectorizer = None
        self.scaler = None

    def load_and_prep(self, filepath):
        logger.info(f"📥 Loading {self.task_type} data from {filepath}...")
        df = pd.read_csv(filepath)

        if self.task_type == 'network':
            # Synthetic Data has columns: feature_0...feature_19, label
            X = df.drop(columns=['label'])
            y = df['label']

            # Scale
            self.scaler = MinMaxScaler()
            X = self.scaler.fit_transform(X)

            # Save Scaler
            joblib.dump(self.scaler, f"{config.MODELS_DIR}network_scaler.pkl")
            return train_test_split(X, y.values, test_size=0.2, random_state=42)

        elif self.task_type == 'web':
            # Synthetic Data has columns: payload, label
            X_text = df['payload'].astype(str)
            y = df['label'].values

            # Vectorize
            self.vectorizer = TfidfVectorizer(max_features=500, analyzer='char', ngram_range=(1,3))
            X = self.vectorizer.fit_transform(X_text).toarray()

            # Save Vectorizer
            joblib.dump(self.vectorizer, f"{config.MODELS_DIR}web_vectorizer.pkl")
            return train_test_split(X, y, test_size=0.2, random_state=42)

class SecurityDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float32)
        self.y = torch.tensor(y, dtype=torch.float32).unsqueeze(1)

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# --- 3. MODELS ---
class NetworkShield(nn.Module):
    def __init__(self, input_dim):
        super(NetworkShield, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

class WebBrain(nn.Module):
    def __init__(self, input_dim):
        super(WebBrain, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# --- 4. TRAINER ---
class EnterpriseTrainer:
    def __init__(self, model, task_name):
        self.model = model.to(config.DEVICE)
        self.task_name = task_name
        self.criterion = nn.BCELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=config.LR)

    def train(self, train_loader, test_loader):
        logger.info(f"🔥 Starting Training for {self.task_name}...")
        self.model.train()

        for epoch in range(config.EPOCHS):
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(config.DEVICE), y_batch.to(config.DEVICE)
                self.optimizer.zero_grad()
                outputs = self.model(X_batch)
                loss = self.criterion(outputs, y_batch)
                loss.backward()
                self.optimizer.step()

            # Simple log per epoch
            logger.info(f"Epoch {epoch+1}/{config.EPOCHS} complete.")

        self.save_model()

    def save_model(self):
        # Save PyTorch Model
        path = f"{config.MODELS_DIR}{self.task_name}_model.pth"
        torch.save(self.model.state_dict(), path)
        logger.info(f"💾 Model saved to {path}")

        # Save ONNX (Optimized)
        try:
            dummy_input = torch.randn(1, next(self.model.parameters()).shape[1], device=config.DEVICE)
            onnx_path = f"{config.MODELS_DIR}{self.task_name}_optimized.onnx"
            torch.onnx.export(self.model, dummy_input, onnx_path)
            logger.info(f"⚡ ONNX Model exported to {onnx_path}")
        except Exception as e:
            logger.warning(f"ONNX Export skipped: {e}")

# --- 5. EXECUTION ---
def run_pipeline():
    # 1. Network Shield
    print("\n--- 🛡️ TRAINING NETWORK SHIELD ---")
    net_factory = DataFactory('network')
    X_train, X_test, y_train, y_test = net_factory.load_and_prep('network_data.csv')

    loader = DataLoader(SecurityDataset(X_train, y_train), batch_size=config.BATCH_SIZE, shuffle=True)
    model = NetworkShield(input_dim=X_train.shape[1])
    EnterpriseTrainer(model, "network_shield").train(loader, loader)

    # 2. Web Brain
    print("\n--- 🧠 TRAINING WEB BRAIN ---")
    web_factory = DataFactory('web')
    X_train_w, X_test_w, y_train_w, y_test_w = web_factory.load_and_prep('web_data.csv')

    loader_w = DataLoader(SecurityDataset(X_train_w, y_train_w), batch_size=config.BATCH_SIZE, shuffle=True)
    model_w = WebBrain(input_dim=X_train_w.shape[1])
    EnterpriseTrainer(model_w, "web_brain").train(loader_w, loader_w)

if __name__ == "__main__":
    run_pipeline()


--- 🛡️ TRAINING NETWORK SHIELD ---
[torch.onnx] Obtain model graph for `NetworkShield([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `NetworkShield([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅

--- 🧠 TRAINING WEB BRAIN ---
[torch.onnx] Obtain model graph for `WebBrain([...]` with `torch.export.export(..., strict=False)`...
[torch.onnx] Obtain model graph for `WebBrain([...]` with `torch.export.export(..., strict=False)`... ✅
[torch.onnx] Run decomposition...
[torch.onnx] Run decomposition... ✅
[torch.onnx] Translate the graph into ONNX...
[torch.onnx] Translate the graph into ONNX... ✅


In [ ]:
!ls -l ./artifacts/

total 588
-rw-r--r-- 1 root root   1959 Jan 24 07:31 network_scaler.pkl
-rw-r--r-- 1 root root  17137 Jan 24 07:31 network_shield_model.pth
-rw-r--r-- 1 root root   8934 Jan 24 07:31 network_shield_optimized.onnx
-rw-r--r-- 1 root root  13312 Jan 24 07:31 network_shield_optimized.onnx.data
-rw-r--r-- 1 root root 259693 Jan 24 07:31 web_brain_model.pth
-rw-r--r-- 1 root root   5834 Jan 24 07:31 web_brain_optimized.onnx
-rw-r--r-- 1 root root 257024 Jan 24 07:31 web_brain_optimized.onnx.data
-rw-r--r-- 1 root root  17055 Jan 24 07:31 web_vectorizer.pkl


In [ ]:
import shutil
from google.colab import files

# Zip the artifacts folder
shutil.make_archive('server_guard_models', 'zip', 'artifacts')

# Trigger download
files.download('server_guard_models.zip')

print("✅ Download started! You should have 'server_guard_models.zip' shortly.")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Download started! You should have 'server_guard_models.zip' shortly.
